In [1]:
import airsim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.interpolate import splprep, splev

from utils import Gate 
from client import Simulator


sim = Simulator()
sim.get_gates(7)
# get gate_2 manually because there is some bug in the api
sim.gates[1] = Gate.from_unreal_position(-300, 7640, 100, yaw=50.0/180*np.pi)
p = sim.track_position_matrix()

# interpolation (3rd degree with some smoothing)
tck, u = splprep(p.T, u=None, s=5.0, per=1, k=3) 
u_new = np.linspace(u.min(), u.max(), 1000)
ev = np.array(splev(u_new, tck, der=0)).T

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

gate_2 was nan


NameError: name 'q' is not defined

In [ ]:
# plot
def plot_gates(xx, yy):
    fig, ax = plt.subplots()
    plt.plot(ev[:,xx], ev[:,yy], 'b--')
    ax.scatter(p[:,xx], p[:,yy])
    for i in range(7):
        ax.annotate(str(i+1), (p[i, xx], p[i, yy]))

In [ ]:
p

In [ ]:
plot_gates(0, 1)
plot_gates(1,2)
plot_gates(0,2)

In [ ]:
# Loop the drone through the track using simSetVehiclePose
import time
client = airsim.VehicleClient()
client.confirmConnection()
i = 0
pose = client.simGetVehiclePose()
while True:
    print(i % len(ev))
    pos = ev[i % len(ev), :]
    print("Sent position:", pos)
    pose.position = airsim.Vector3r(pos[0], pos[1], pos[2])
    success = client.simSetVehiclePose(pose, True)
    time.sleep(0.01)
    i += 1